# Capstone Project Final : Best location for opening a Lebanese restaurant in Paris



## Table of contents
* [1 - Introduction: Business Problem](#introduction)
* [2 - Data Description to solve the problem](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1 - Introduction: Business Problem <a name="introduction"></a>

The goal of the project is to open a lebanese Restaurant in Paris, so the business problem, is to find the best place for opening the restaurant.
The target will be a investor who want to opening a restaurant, and to help him to find the best place base on Data Science.

The main obejective is to find the best neighborhood for opening based on these criterias :
* Attractivity of the area
* no competition (no other Lebanese restaurant nearby)
* great potential ( People use to or like eating lebanese food)
* Area with lot of restaurant


The fist step will be to find the top neighborhood where you can find lebanese restaurant.
The second will be to rate them, to find the best neighborhood.
And finally to find a similar neighborhood with no lebanese restaurant, for opening e new one.




## 2 - Data Description to solve the problem <a name="data"></a>

For this Business problem, we will use the API FourSquare for :

* Get and analyse the venues of Paris
* Finding the neighborhoods, where you can find Lebanese restaurant
* Analyse each fo these neighborhood to find the best one, based on these criterias : Good revenues, Attractive area, good reviews etc...
* Find a similar neighborhood, but with no lebanese restaurant opened yet.


    
    

### Restaurant analysis in Paris

We will import the useful livbraries and use FourQuare API for the analysis

In [4]:
# Import Libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup #Library for web scrapping
import urllib.request 

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Use geopy library to get the latitude and longitude values of PAris

In [6]:

address = 'Paris'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566101, 2.3514992.


#### Define Foursquare Credentials and Version


In [8]:

CLIENT_ID = 'UQORBT4J0DMW2MWBCIFHZHJM3KJA1G0ZZOBAHNXVTNDOYWYO' # your Foursquare ID
CLIENT_SECRET = '5VBPSSBVFQFFMILW50QUEK2GHIBD05ZIFNA1YUW10TOK01N5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 

Your credentails:
CLIENT_ID: UQORBT4J0DMW2MWBCIFHZHJM3KJA1G0ZZOBAHNXVTNDOYWYO
CLIENT_SECRET:5VBPSSBVFQFFMILW50QUEK2GHIBD05ZIFNA1YUW10TOK01N5


#### Explore Neighborhoods in Paris
Let's create a function to repeat the same process to all the neighborhoods in Paris

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)